In [5]:
!pip install selenium
!apt-get update # to update ubuntu to correctly run apt install
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
import sys
import pandas as pd
import time
import json
from sqlalchemy import create_engine, MetaData, Table, Column, Integer, String

sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

Hit:1 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Hit:3 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:5 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:6 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:7 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:8 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:9 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Ign:10 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:11 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:12 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:13 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64

# **To  create the Database**

In [6]:
engine = create_engine('sqlite:///Products.db', echo=True)
meta = MetaData()

products = Table(
    "products", meta,
    Column("id", Integer, primary_key=True),
    Column("Title", String, nullable=False),
    Column("img_URL", String),
    Column("Price", String),
    Column("Product_Details", String),
)

## **To create the table uncomment the code and run it**

In [7]:
# meta.create_all(engine) # # commented after creating the tables

2022-02-16 05:02:58,707 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-02-16 05:02:58,710 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("products")
2022-02-16 05:02:58,714 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-02-16 05:02:58,717 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("products")
2022-02-16 05:02:58,719 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-02-16 05:02:58,723 INFO sqlalchemy.engine.Engine 
CREATE TABLE products (
	id INTEGER NOT NULL, 
	"Title" VARCHAR NOT NULL, 
	"img_URL" VARCHAR, 
	"Price" VARCHAR, 
	"Product_Details" VARCHAR, 
	PRIMARY KEY (id)
)


2022-02-16 05:02:58,725 INFO sqlalchemy.engine.Engine [no key 0.00258s] ()
2022-02-16 05:02:58,740 INFO sqlalchemy.engine.Engine COMMIT


# **Bot that scrape the required details of the products and adds it to the Database**

In [9]:
# # Change the path according to where the chrome webdriver is installed on your system

bot = webdriver.Chrome('chromedriver',chrome_options=chrome_options)

dicts_list = []

data = pd.read_csv("Amazon Scraping - Sheet1.csv")
# # change the range to your choice to scrape the no of URLS you want
for i in range(0, 3):
    pdt_title = ""
    pdt_img_url = ""
    pdt_price = ""
    pdt_description = []
    row = data.iloc[i]
    country = row.country
    asin = row.Asin
    URL = f"https://www.amazon.{country}/dp/{asin}"

    bot.get(URL)

    time.sleep(2)

    try:
        cookies_accept = bot.find_element("name", "accept")
        cookies_accept.click()

    except NoSuchElementException:
        try:
            time.sleep(2)
            title = bot.find_element("id", "productTitle")
        except NoSuchElementException:
            print(f"{URL} not available")
            time.sleep(2)
            pass
        else:
            pdt_title = title.text

            time.sleep(2)

            img_url = bot.find_element("css selector", "div#leftCol div#main-image-container img")
            pdt_img_url = img_url.get_attribute("src")
            time.sleep(2)

            try:
                price = bot.find_element("css selector", "div#centerCol div#formats a span.a-color-base")
                time.sleep(2)
            except NoSuchElementException:
                price = bot.find_element(
                    "css selector",
                    "div#centerCol div#corePriceDisplay_desktop_feature_div span.priceToPay span.a-price"
                )
                pdt_price = price.text
                time.sleep(2)
            else:
                if not price:
                    price = bot.find_element(
                        "css selector",
                        "div#centerCol div#corePriceDisplay_desktop_feature_div span.a-price"
                    )
                    pdt_price = price.text
                    time.sleep(2)
                else:
                    pdt_price = price.text

            try:
                description = bot.find_elements("css selector",
                                                "div#detailBullets_feature_div  div#detailBullets_feature_div li")
                time.sleep(2)
            except NoSuchElementException:
                description = bot.find_elements("css selector", "div#feature-bullets span.a-list-item")
                time.sleep(2)
                pdt_description = [item.text for item in description]
            else:
                if not description:
                    description = bot.find_elements("css selector", "div#feature-bullets span.a-list-item")

                    pdt_description = [item.text for item in description]
                else:
                    pdt_description = [item.text for item in description]

            time.sleep(2)

    else:
        time.sleep(4)
        title = bot.find_element("id", "productTitle")
        pdt_title = title.text

        time.sleep(1)

        img_url = bot.find_element("css selector", "div#leftCol div#main-image-container img")
        pdt_img_url = img_url.get_attribute("src")
        time.sleep(2)

        try:
            price = bot.find_element("css selector", "div#centerCol div#formats a span.a-color-base")
            time.sleep(2)
        except NoSuchElementException:
            price = bot.find_element(
                "css selector",
                "div#centerCol div#corePriceDisplay_desktop_feature_div span.a-price"
            )
            pdt_price = price.text
            time.sleep(2)
        else:
            if not price:
                price = bot.find_element(
                    "css selector",
                    "div#centerCol div#corePriceDisplay_desktop_feature_div span.a-price"
                )
                pdt_price = price.text
                time.sleep(2)
            else:
                pdt_price = price.text

        try:
            description = bot.find_elements("css selector",
                                            "div#detailBullets_feature_div  div#detailBullets_feature_div li")
            time.sleep(2)
        except NoSuchElementException:
            description = bot.find_elements("css selector", "div#feature-bullets span.a-list-item")
            time.sleep(2)
            pdt_description = [item.text for item in description]
        else:
            if not description:
                description = bot.find_elements("css selector", "div#feature-bullets span.a-list-item")

                pdt_description = [item.text for item in description]
            else:
                pdt_description = [item.text for item in description]

        time.sleep(2)

    data_dict = {
        "Title": pdt_title,
        "Img_URL": pdt_img_url,
        "Price": pdt_price,
        "Product Details": pdt_description,
    }

    if pdt_title != "":
        dicts_list.append(data_dict)

        add_to_db = products.insert().values(
            Title=pdt_title,
            img_URL=pdt_img_url,
            Price=pdt_price,
            Product_Details="\n".join(pdt_description)
        )
        conn = engine.connect()
        result = conn.execute(add_to_db)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: DeprecationWarning: use options instead of chrome_options
  This is separate from the ipykernel package so we can avoid doing imports until


https://www.amazon.de/dp/1015 not available
https://www.amazon.fr/dp/1015 not available
2022-02-16 05:05:00,902 INFO sqlalchemy.engine.Engine INSERT INTO products ("Title", "img_URL", "Price", "Product_Details") VALUES (?, ?, ?, ?)
2022-02-16 05:05:00,904 INFO sqlalchemy.engine.Engine [generated in 0.00177s] ('Old Spice Shaving Cream - 70 g (original) - Pack of 2', 'https://m.media-amazon.com/images/I/41xHbjdSXJL._SX355_PIbundle-2,TopRight,0,0_SX355SY222SH20_.jpg', '€8\n00', 'Good lubrication quality in Old Spice Lather shaving foam gives optimal lubricity actions a closer comfortable shave. Rich, longer - lasting foam holds during your shave. Use Old Spice Lather shaving cream where you will find the masculine freshness of old spices.')
2022-02-16 05:05:00,909 INFO sqlalchemy.engine.Engine COMMIT


# **To create a output Json file** 

In [10]:
json_object = json.dumps({"data": dicts_list}, indent=4)

with open("output.json", "w") as file:
    file.write(json_object)